In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import Toolkit_01 as ml

# Step 1: Load the data

In [24]:
init_data = pd.read_csv("AdmissionPredict.csv")
data = init_data.copy().dropna()
data_x = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

array([[337.  , 118.  ,   4.  , ...,   4.5 ,   9.65,   1.  ],
       [314.  , 103.  ,   2.  , ...,   3.  ,   8.21,   0.  ],
       [330.  , 115.  ,   5.  , ...,   3.  ,   9.34,   1.  ],
       ...,
       [330.  , 116.  ,   4.  , ...,   4.5 ,   9.45,   1.  ],
       [312.  , 103.  ,   3.  , ...,   4.  ,   8.78,   0.  ],
       [333.  , 117.  ,   4.  , ...,   4.  ,   9.66,   1.  ]])

# Step 2: Split the data into training and validation sets

In [4]:
data_train, data_test, y_train, y_test = ml.split_scale(data_x, y, test_size=0.2, random_state=1)

# Step 3: Define the MLP

In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Step 4: Define the hyperparameters

In [28]:
print(data_x.shape[1])  # it is 7 here
input_size = 7
hidden_size = 3
output_size = 1
num_epochs = 100
batch_size = 100
learning_rate = 0.001

# Step 5: Define Data Loaders

In [29]:
temp_x = torch.Tensor(data_train)
temp_y = torch.Tensor(y_train)
train_data = torch.utils.data.TensorDataset(temp_x, temp_y)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [30]:
temp_x = torch.Tensor(data_test)
temp_y = torch.Tensor(y_test)
test_data = torch.utils.data.TensorDataset(temp_x, temp_y)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Step 6: Train the MLP

In [31]:
model = MLP(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        output = model(inputs)
        loss = criterion(output, labels.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in test_loader:
            output = model(inputs)
            val_loss += criterion(output, labels.unsqueeze(1)).item() * inputs.size(0)
        val_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}")


Epoch 1/100, Training Loss: 1.0359, Validation Loss: 0.2738
Epoch 2/100, Training Loss: 1.0667, Validation Loss: 0.2478
Epoch 3/100, Training Loss: 0.8569, Validation Loss: 0.2220
Epoch 4/100, Training Loss: 0.6369, Validation Loss: 0.1962
Epoch 5/100, Training Loss: 0.7945, Validation Loss: 0.1714
Epoch 6/100, Training Loss: 0.5014, Validation Loss: 0.1493
Epoch 7/100, Training Loss: 0.6136, Validation Loss: 0.1305
Epoch 8/100, Training Loss: 0.4076, Validation Loss: 0.1155
Epoch 9/100, Training Loss: 0.1559, Validation Loss: 0.1041
Epoch 10/100, Training Loss: 0.3344, Validation Loss: 0.0954
Epoch 11/100, Training Loss: 0.3649, Validation Loss: 0.0889
Epoch 12/100, Training Loss: 0.5345, Validation Loss: 0.0836
Epoch 13/100, Training Loss: 0.3222, Validation Loss: 0.0790
Epoch 14/100, Training Loss: 0.1389, Validation Loss: 0.0748
Epoch 15/100, Training Loss: 0.3019, Validation Loss: 0.0709
Epoch 16/100, Training Loss: 0.1733, Validation Loss: 0.0668
Epoch 17/100, Training Loss: 0.25

In [34]:
new_data = torch.Tensor([[340, 120, 5, 4, 4, 9, 1]])
prediction = model(new_data).item()
print(f"Predicted chance of admission: {prediction:.4f}")
new_data = torch.Tensor([[315, 110, 5, 4, 4, 9, 1]])
prediction = model(new_data).item()
print(f"Predicted chance of admission: {prediction:.4f}")
new_data = torch.Tensor([[290, 80, 1, 1, 1, 1, 0]])
prediction = model(new_data).item()
print(f"Predicted chance of admission: {prediction:.4f}")

Predicted chance of admission: 0.9039
Predicted chance of admission: 0.7877
Predicted chance of admission: 0.4946
